In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

# TODO: 
- [ ] word embedding multiply tfidf
- [ ] use word2vec
- [ ] k-fold cv

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
val_file = 'example/val.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, val_file, test_file, pretrained_file, config)

In [4]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [5]:
AttractiveData.TEXT.vocab.freqs.most_common(15)

[('the', 1524),
 ('to', 1180),
 ('a', 940),
 ('of', 903),
 ('and', 884),
 ('in', 843),
 ('for', 517),
 ('with', 383),
 ('on', 376),
 ('as', 337),
 ('is', 328),
 ('after', 261),
 ('at', 249),
 ('from', 233),
 ('by', 217)]

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [7]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 8
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 150
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 6e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.1
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

In [9]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (bigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
   )
   (trigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
   )
   (encoder_bigram_first): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (encoder_bigram_second): LSTM(60, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (encoder_trigram_first): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (encoder_trigram_second): LSTM(60, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=240, out_features=30, bias=True)
     (

In [10]:
AttractiveTrainer.train()

      | 5/150 [00:02<01:22,  1.76it/s]
EP_train | train loss: 0.5718166687909294 | val loss: 0.731521583070942 |
====
Epoch:   4%|▍         | 6/150 [00:03<01:22,  1.74it/s]
EP_train | train loss: 0.5606825566759297 | val loss: 0.6988431776271147 |
====
Epoch:   5%|▍         | 7/150 [00:03<01:21,  1.76it/s]
EP_train | train loss: 0.5599328742307775 | val loss: 0.6885443481744504 |
====
Epoch:   5%|▌         | 8/150 [00:04<01:20,  1.76it/s]
EP_train | train loss: 0.5589881392086253 | val loss: 0.6809147596359253 |
====
Epoch:   6%|▌         | 9/150 [00:05<01:20,  1.75it/s]
EP_train | train loss: 0.5606503271589092 | val loss: 0.6525255535163131 |
====
Epoch:   7%|▋         | 10/150 [00:05<01:20,  1.74it/s]
EP_train | train loss: 0.5512689085567699 | val loss: 0.6565861701965332 |
====
Epoch:   7%|▋         | 11/150 [00:06<01:22,  1.69it/s]
EP_train | train loss: 0.5448641814437567 | val loss: 0.6211169140011656 |
====
Epoch:   8%|▊         | 12/150 [00:06<01:23,  1.66it/s]
EP_train | tra

## for classification, not better

## Below is testing

In [8]:
from attractivenet import AttractiveNet

PATH = './model/CNN_LSTM_20201104-150343_0.3549.85'

load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (encoder_bigram_first): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_bigram_second): LSTM(60, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_trigram_first): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_trigram_second): LSTM(60, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=240, out_features=30, bias=True)
    (1): ReLU()
    (2): Lin

In [9]:
def predict_attractive(sentence, category, phase):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    return prediction

In [10]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below for statistics

In [11]:
import statistics
from sklearn.metrics import mean_squared_error

In [12]:
# # train mean = 3.15, test mean = 2.8
# train_list = []
# for i, sentence in enumerate(AttractiveData.train_data):
#     prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
#     train_list.append(prediction.item())
#     # train_list.append(prediction.item())
# # print(train_list)
# mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

In [13]:
# train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

In [14]:
# a = AttractiveData.df_train['Label'].to_list()
# statistics.mean(a), statistics.stdev(a)

In [15]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.849695928296329, 0.3770257875481673)

In [16]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.018452828976924473, 2.832396388579045, 0.3599597701326745)

In [18]:
mean_squared_error(pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list(), pd.read_csv('./predict/ensemble.csv').sort_values(['ID']).Label.to_list())

0.1609754968529207